In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
with open(r'C:\Users\10634\Desktop\recluster20230828\全qu模式\GPT重命名后数据\52544费用相关问题..json','r',encoding='utf-8') as f:
    data = json.load(f)
data

{'136': {'新类别': '优惠多少钱？',
  '占比': '0.36%',
  '问题': ['四千六百八原价，那有优惠吗？',
   '能优惠多少钱。',
   '如果优惠多，优惠多少钱，像。',
   '优惠活动什么时候都有这套路，行行行，先这样。',
   '就是活动优惠后的价格是多少。',
   '那优惠多能多少钱呢，给我低一个。',
   '报是有优惠吗？',
   '优惠多少钱？',
   '你优惠多少钱呢？',
   '那你什么优惠价呢？',
   '优惠要多少钱？',
   '优惠，优惠价是多少钱？',
   '优惠下来是多少钱？',
   '可以优惠多少呢。',
   '那你京东那个。优惠券那个多，能优惠多少那个。',
   '什么？优惠额是多少钱来着。',
   '优惠这个，这个是费用多少。',
   '优惠下来是多少呢？',
   '什么优惠吗？对。',
   '优惠的话是怎么优惠。',
   '不是泛优质旗舰店吗？还是说优惠吗。',
   '那个优惠下来多少钱呢？',
   '你报这个是最优惠的价格吗？',
   '下优惠是多少？',
   '之后交多少钱？最优惠的情况下，你帮。',
   '这个优惠券是怎么优惠的。',
   '对实他班他优惠多少。',
   '优惠多少钱？',
   '班的话，它有什么优惠活动吗？',
   '有一个优惠券是不是。',
   '有优惠吗？就是优惠多少。',
   '这个费用下来最低多少钱优惠。',
   '五是总是要交多少钱。优惠。',
   '也就是说，那最优惠的价格是多少？',
   '这样，那你这边可有什么优惠呢。',
   '报名是多少钱的，优惠是优惠。',
   '优惠券可不可以？']},
 '132': {'新类别': '',
  '占比': '0.69%',
  '问题': ['像你就先告诉我得多少得多少费用。',
   '那你先和我说的费用多少钱？你长话短说。',
   '你就说费用吧，我现在这边有客人等在这里，你说一下这个费用是多少。',
   '我先问问你多少钱，我给他们咨询的。',
   '费用，我马上进电梯了，你说的费用多少？',
   '行，我知道。这个费用是，这个费用是多少。',
   '费用是多少，你给我说。',
   '的什么费用？是什么价位什么的给我。',
   

In [22]:
def sorted_per(data):
    if data != {}:
        use_per_list = []
        for i in data.values():
            per = float(i['占比'][:-1])/100
            use_per_list.append(per)
        sorted_data = {}
        df_use = pd.concat([pd.Series(list(data.keys())),pd.Series(use_per_list)],axis=1)
        df_use.columns = ['id','per']
        df_use = df_use.sort_values(by='per',ascending=False)
        for i in list(df_use['id']):
            sorted_data[str(i)] = data[str(i)]
        return sorted_data
    else:
        return None
def sorted_dic(data_use,data_filter):
    if data_filter != None:
        use_per_list,filter_per_list = [], []
        for i in data_use.values():
            per = float(i['占比'][:-1])/100
            use_per_list.append(per)
        for i in data_filter.values():
            per = float(i['占比'][:-1])/100
            filter_per_list.append(per)
        sorted_data_use, sorted_data_filter = {}, {}
        df_use = pd.concat([pd.Series(list(data_use.keys())),pd.Series(use_per_list)],axis=1)
        df_use.columns = ['id','per']
        df_use = df_use.sort_values(by='per',ascending=False)
        df_filter = pd.concat([pd.Series(list(data_filter.keys())),pd.Series(filter_per_list)],axis=1)
        df_filter.columns = ['id','per']
        df_filter = df_filter.sort_values(by='per',ascending=False)
        for i in list(df_use['id']):
            sorted_data_use[str(i)] = data_use[str(i)]
        for j in list(df_filter['id']):
            sorted_data_filter[str(j)] = data_filter[str(j)]
        return sorted_data_use, sorted_data_filter


def filter1(data, sortflag=0):
    filter_id1 = []
    use_id1 = []
    for keys1,values1 in data.items():
        error = ['无法总结', '无法总结。', '总结出错', [], '', '无法总结.']
        # 第一轮过滤
        if values1['新类别'] not in error:
            use_id1.append(keys1)
        else:
            filter_id1.append(keys1)
    data_use1 = {}
    data_filter1 = {}
    for i1 in use_id1:
        data_use1[str(i1)] = data[str(i1)]
    for j1 in filter_id1:
        data_filter1[str(j1)] = data[str(j1)]
    if sortflag == 0:
        return data_use1, data_filter1
    elif sortflag == 1:
        return sorted_per(data_use1),sorted_per(data_filter1)


def filter2(data, sortflag=0,limit=0.9):
    if len(list(data.keys())) > 10:
        s = SentenceTransformer('shibing624/text2vec-base-chinese')
        new_class_list = [i['新类别'] for i in data.values()]
        new_class_id_list = list(data.keys())
        df = pd.DataFrame(new_class_list)
        df['1'] = df.iloc[:, 0]
        df.columns = ['0', '1']
        df.index = new_class_id_list
        df_sim = pd.DataFrame([])
        for (new_class, class_id) in zip(list(df['0']), df.index):
            veci = s.encode(new_class)
            new_class_sim_list = []
            for j in list(df['1']):
                vecj = s.encode(j)
                sim = cosine_similarity(veci.reshape(1, -1), vecj.reshape(1, -1))
                sim = sim[0,0]
                new_class_sim_list.append(sim)
            df_sim[class_id] = pd.Series(new_class_sim_list)
        df_sim.index = df.index
        index = np.where(df_sim > limit)
        index = [*zip(index[0], index[1])]
        index1 = []
        for i in index:
            if i[0] != i[1]:
                index1.append(i)
        index1 = [list(i) for i in index1]
        use_index = []
        for i in range(len(index1)):
            rev_list = []
            for ind in index1[i + 1:]:
                rev_list.append(list(reversed(ind)))
            if index1[i] in rev_list and index1[i] not in use_index:
                use_index.append(index1[i])
        merge_class_list = []
        for i in use_index:
            merge_class_list.append([df_sim.index[i[0]], df_sim.columns[i[1]]])
        merge_list = []
        merged_list = []
        # keys = list(data.keys())  # 获取字典中的键列表
        use_id2 = []
        filter_id2 = []
        if merge_class_list != []:
            for col_ind in merge_class_list:
                if col_ind[0] not in merged_list and col_ind[0] not in merge_list:
                    if col_ind[0] in data and col_ind[1] in data:  # 检查键是否存在于 data 中
                        merge_per = (float(data[col_ind[0]]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1]))
                        merge_qu_list = [data[col_ind[0]]['问题'], data[col_ind[1]]['问题']]
                        merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
                        data[col_ind[0] + ' ' + col_ind[1]] = {'新类别': data[col_ind[0]]['新类别'],
                                                               '占比': "%.2f%%" % (merge_per),
                                                               '问题': merge_qu_list}
                        # data.pop(col_ind[0])
                        # data.pop(col_ind[1])
                        filter_id2.append(col_ind[0])
                        filter_id2.append(col_ind[1])
                        merge_list.append(col_ind[0])
                        merged_list.append(col_ind[1])
                elif col_ind[0] in merged_list:
                    keys = list(data.keys())
                    ind = merged_list.index(col_ind[0])
                    merge = merge_list[ind]
                    for i in keys:
                        if merge in str(i).split(' ') and col_ind[1] in data:  # 检查键是否存在于 data 中
                            merge_per = float(data[i]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1])
                            merge_qu_list = [data[i]['问题'], data[col_ind[1]]['问题']]
                            merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
                            data[i + ' ' + col_ind[1]] = {'新类别': data[i]['新类别'],
                                                           '占比': "%.2f%%" % (merge_per),
                                                           '问题': merge_qu_list}
                            # data.pop(i)
                            # data.pop(col_ind[1])
                            filter_id2.append(i)
                            filter_id2.append(col_ind[1])
                            merge_list.append(merge)
                            merged_list.append(col_ind[1])
                elif col_ind[0] in merge_list:
                    keys = list(data.keys())
                    for i in keys:
                        if col_ind[0] in str(i).split(' ') and col_ind[1] in data:  # 检查键是否存在于 data 中
                            merge_per = float(data[i]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1])
                            merge_qu_list = [data[i]['问题'], data[col_ind[1]]['问题']]
                            merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
                            data[i + ' ' + col_ind[1]] = {'新类别': data[i]['新类别'],
                                                           '占比': "%.2f%%" % (merge_per),
                                                           '问题': merge_qu_list}
                            # data.pop(i)
                            # data.pop(col_ind[1])
                            filter_id2.append(i)
                            filter_id2.append(col_ind[1])
                            merge_list.append(col_ind[0])
                            merged_list.append(col_ind[1])
            filter_id2 = list(set(filter_id2))
            for i in list(data.keys()):
                for j in filter_id2:
                    if i != j:
                        print(i)
                        use_id2.append(i)
            data_use2 = {}
            data_filter2 = {}
            for i in use_id2:
                data_use2[str(i)] = data[str(i)]
            for j in filter_id2:
                data_filter2[str(j)] = data[str(j)]
        
            if sortflag == 0:
                return data_use2, data_filter2
            if sortflag == 1:
                return sorted_per(data_use2), sorted_per(data_filter2)
        else:
            if sortflag == 0:
                return data,{}
            if sortflag == 1:
                return sorted_per(data),{}
    else:
        if sortflag == 0:
            return data, {}
        if sortflag == 1:            
            return sorted_per(data), {}

def filter3(data,raw_class,sortflag=0):
    use_id3 = []
    filter_id3 = []
    s = SentenceTransformer('shibing624/text2vec-base-chinese')
    raw_vec = s.encode(raw_class)
    for keys3, values3 in data.items():
        new_class3 = values3['新类别']
        new_vec3 = s.encode(new_class3)
        sim3 = cosine_similarity(raw_vec.reshape(1, -1), new_vec3.reshape(1, -1))
        sim3 = sim3[0,0]
        if sim3 > 0.8:
            filter_id3.append(keys3)
        else:
            use_id3.append(keys3)
    data_use3,data_filter3 = {}, {}
    for i in use_id3:
        data_use3[str(i)] = data[str(i)]
    for j in filter_id3:
        data_filter3[str(j)] = data[str(j)]

    if sortflag == 0:
        return data_use3, data_filter3
    if sortflag == 1:
        return sorted_per(data_use3),sorted_per(data_filter3)

def filter4(data, num=15, sortflag=0):
    per_list4 = []
    for keys4, values4 in data.items():
        per4 = values4['占比']
        per4 = float(per4[:-1]) / 100
        per_list4.append(per4)
    df4 = pd.concat([pd.Series(data.keys()), pd.Series(per_list4)], axis=1)
    df4.columns = ['id', 'per']
    df4 = df4.sort_values(by='per', ascending=False)
    if len(df4['id']) > num:
        use_id4 = list(df4['id'])[:num]
        del4 = list(df4['id'])[num-1:]
        data_use4 = {}
        data_filter4 = {}
        for i in use_id4:
            data_use4[str(i)] = data[str(i)]
        for i in del4:
            data_filter4[str(i)] = data[str(i)]
    if sortflag == 0:
        return data_use4, data_filter4
    if sortflag == 1:
        return sorted_per(data_use4),sorted_per(data_filter4)
    else:
        data_use4 = {}
        use_id4 = list(df4['id'])[:num]
        for i in use_id4:
            data_use4[str(i)] = data[str(i)]

    if sortflag == 0:
        return data_use4, None
    if sortflag == 1:
        return sorted_per(data_use4), None

def qu_filter(data,top=10):
    s = SentenceTransformer('shibing624/text2vec-base-chinese')
    data_use = {}
    for keys, values in data.items():
        dic = {}
        qu_vec_list = []
        new_class_vec = s.encode(values['新类别'])
        for i in values:
            qu_vec = s.encode(i)
            qu_vec_list.append(qu_vec)
        sim_list = []
        for i in qu_vec_list:
            sim = cosine_similarity(new_class_vec.reshape(1, -1), i.reshape(1, -1))
            sim = float(sim)
            sim_list.append(sim)
        df = pd.concat([pd.Series(list(values['问题'])), pd.Series(sim_list)], axis=1)
        df.columns = ['qu', 'sim']
        df = df.sort_values(by='sim', ascending=False)
        dic['新类别'] = values['新类别']
        dic['占比'] = values['占比']
        if len(df['qu']) <= top:
            dic['问题'] = list(df['qu'])[:top]
            data_use[keys] = dic
        else:
            dic['问题'] = list(df['qu'])
            data_use[keys] = dic
    return data_use

In [45]:
df[0]

136                 优惠多少钱？
89              询问是否还有优惠空间
7      报名费用、学习时间、考试费用等问题总结
14           线上教育费用最低价是多少？
180               需要交多少费用？
              ...         
174             第一个班学费价格查询
57                  班型价格一览
142                 费用情况总结
166               快递费用如何计算
156         续费手续费及费用收取方式汇总
Name: 0, Length: 179, dtype: object

In [46]:
du1,df1 = filter1(data)
s = SentenceTransformer('shibing624/text2vec-base-chinese')
new_class_list = [i['新类别'] for i in du1.values()]
df = pd.DataFrame(new_class_list)
df.index = du1.keys()
df_sim = pd.DataFrame([])
for (i,id) in zip(list(df[0]),df.index):
    veci = s.encode(i)
    sim_list = []
    for j in list(df[0]):
        vecj = s.encode(j)
        sim = cosine_similarity(veci.reshape(1,-1),vecj.reshape(1,-1))
        sim = sim[0,0]
        sim_list.append(sim)
    df_sim[id] = sim_list
df_sim


KeyboardInterrupt



In [28]:
new_id_class_list = []
for keys,values in data.items():
    error = ['无法总结','没有办法总结','总结出错',[],'','无法总结。']
    # 第一轮过滤
    if values['新类别'] not in error:
        new_id_class_list.append([keys,values['新类别']])
    # 第二轮过滤:过滤新类别两两之间相似度比较高的
s = SentenceTransformer('shibing624/text2vec-base-chinese')
new_class_list = [i[1] for i in new_id_class_list]
df = pd.DataFrame(new_class_list)
df['1'] = df.iloc[:,0]
df.columns = ['0','1']
df.index = [i[0] for i in new_id_class_list]

df_sim = pd.DataFrame([])


for (new_class, class_id) in zip(list(df['0']), df.index):
    sim_i = s.encode(new_class)
    new_class_sim_list = []
    for j in list(df['1']):
        sim_j = s.encode(j)
        sim = cosine_similarity(sim_i.reshape(1,-1),sim_j.reshape(1,-1))
        new_class_sim_list.append(float(sim))
    df_sim[class_id] = pd.Series(new_class_sim_list)
df_sim.index = df.index
index = np.where(df_sim > 0.9)
index = [*zip(index[0],index[1])]
index1 = []
for i in index:
    if i[0] != i[1]:
        index1.append(i)
index1 = [list(i) for i in index1]
use_index = []
for i in range(len(index1)):
    rev_list = []
    for ind in index1[i+1:]:
        rev_list.append(list(reversed(ind)))
    if index1[i] in rev_list and index1[i] not in use_index:
        use_index.append(index1[i]) 
merge_class_list = []
for i in use_index:
    merge_class_list.append([df_sim.index[i[0]],df_sim.columns[i[1]]])

KeyboardInterrupt: 

In [20]:
merge_class_list

[]

# 将新聚类别中相似度大于0.9的类别合并

In [85]:
merge_list = []
merged_list = []
keys = list(data.keys())  # 获取字典中的键列表
use_id2 = []
filter_id2 = []
for col_ind in merge_class_list:
    if col_ind[0] not in merged_list and col_ind[0] not in merge_list:
        if col_ind[0] in data and col_ind[1] in data:  # 检查键是否存在于 data 中
            merge_per = (float(data[col_ind[0]]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1]))
            merge_qu_list = [data[col_ind[0]]['问题'], data[col_ind[1]]['问题']]
            merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
            data[col_ind[0] + ' ' + col_ind[1]] = {'新类别': data[col_ind[0]]['新类别'],
                                                   '占比': "%.2f%%" % (merge_per),
                                                   '问题': merge_qu_list}
#             data.pop(col_ind[0])
#             data.pop(col_ind[1])
            filter_id2.append(col_ind[0])
            filter_id2.append(col_ind[1])
            merge_list.append(col_ind[0])
            merged_list.append(col_ind[1])
    elif col_ind[0] in merged_list:
        keys = list(data.keys())
        ind = merged_list.index(col_ind[0])
        merge = merge_list[ind]
        for i in keys:
            if merge in str(i).split(' ') and col_ind[1] in data:  # 检查键是否存在于 data 中
                merge_per = float(data[i]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1])
                merge_qu_list = [data[i]['问题'], data[col_ind[1]]['问题']]
                merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
                data[i + ' ' + col_ind[1]] = {'新类别': data[i]['新类别'],
                                               '占比': "%.2f%%" % (merge_per),
                                               '问题': merge_qu_list}
#                 data.pop(i)
#                 data.pop(col_ind[1])
                filter_id2.append(i)
                filter_id2.append(col_ind[1])
                merge_list.append(merge)
                merged_list.append(col_ind[1])
    elif col_ind[0] in merge_list:
        keys = list(data.keys())
        for i in keys:
            if col_ind[0] in str(i).split(' ') and col_ind[1] in data:  # 检查键是否存在于 data 中
                merge_per = float(data[i]['占比'][:-1]) + float(data[col_ind[1]]['占比'][:-1])
                merge_qu_list = [data[i]['问题'], data[col_ind[1]]['问题']]
                merge_qu_list = [item for qu_list in merge_qu_list for item in qu_list]
                data[i + ' ' + col_ind[1]] = {'新类别': data[i]['新类别'],
                                               '占比': "%.2f%%" % (merge_per),
                                               '问题': merge_qu_list}
#                 data.pop(i)
#                 data.pop(col_ind[1])
                filter_id2.append(i)
                filter_id2.append(col_ind[1])
                merge_list.append(col_ind[0])
                merged_list.append(col_ind[1])

In [88]:
x1 = data['1']
x2 = data['2']
x11 = x1['新类别']
x22 = x2['新类别']
s = SentenceTransformer('shibing624/text2vec-base-chinese')
vec1 = s.encode(x11)
vec2 = s.encode(x22)
sim = cosine_similarity(vec1.reshape(1,-1),vec2.reshape(1,-1))
float(sim)

0.4379436671733856

# 计算新类别和原类别的相似度，过滤相似度大于0.8的新类别

In [71]:
print(len(data.keys()))
# 第三轮过滤
raw_class = '第一个班型的课程时长及时间'
raw_vec = s.encode(raw_class)
del3 = []
for keys3,values3 in data.items():
    new_class3 = values3['新类别']
    new_vec3 = s.encode(new_class3)
    sim3 = cosine_similarity(raw_vec.reshape(1,-1),new_vec3.reshape(1,-1))
    sim3 = float(sim3)
    if sim3 > 0.8:
        del3.append(keys3)
for i in del3:
    data.pop(i)
print(len(data.keys()))

37
33


In [73]:
# 第四轮过滤
per_list4 = []
for keys4,values4 in data.items():
    per4 = values4['占比']
    per4 = float(per4[:-1])/100
    per_list4.append(per4)
df4 = pd.concat([pd.Series(data.keys()),pd.Series(per_list4)],axis=1)
df4.columns = ['class_id','per']
df4 = df4.sort_values(by='per',ascending=False)
use_id4= list(df4['class_id'])[:15]
del4 = list(df4['class_id'])[14:]
data4_use = {}
data4_filter = {}
for i in use_id4:
    data4_use[str(i)] = data[str(i)]
for i in del4:
    data4_filter[str(i)] = data[str(i)]

In [74]:
data4_filter

{'34': {'新类别': '如何上课？',
  '占比': '2.08%',
  '问题': ['就是那上课是怎么上的。',
   '怎么上课的？',
   '怎么上课的吧？',
   '那这个上课怎么办？',
   '他这个上课是上课是怎么上课的。',
   '先发一个视频看一下。他怎么上课的？',
   '课时是怎么上的？',
   '那要上那个课时要怎么上呢？',
   '那这个课时是怎样学习。',
   '你们机构，你们这个机构是怎么上课的。',
   '这个课时是怎么个上法？',
   '就是说这个课时学习要怎么学习。',
   '你们那个上课是怎么上的？',
   '就是你们那个上课是怎么上的。',
   '那他这个课是怎么上的。',
   '是怎么样上课什么。',
   '然后他上课是怎么上法。',
   '就想学一下你们这个课，你们这个课怎么学。',
   '就是每天上课怎么样上。',
   '那意思，那个课时我怎么上？',
   '不太了解怎么上课。',
   '你说你那个上学那个上课怎么上。咱们。',
   '上课是怎么上得呢',
   '那他上课怎么样。',
   '对，然后怎么他是怎么上课，对。',
   '你说上课是学的什么？',
   '这样上课呢？',
   '对怎么才能出来上课这样子。',
   '我们上课怎么上呢？',
   '你们怎么上课？',
   '怎么上课怎么上，咱们。',
   '上课怎么上呢，咱。',
   '上课怎么上呢',
   '上课怎么上',
   '上课怎么上？',
   '就是这个，这么多课，学生怎么上课。',
   '说我知道怎么上课。',
   '那上上那个课怎么上呢。',
   '以及怎么上课。这个。',
   '我听课怎么个听法。',
   '那我怎么上课？',
   '那上课怎么看呢，上课。',
   '每个。上课怎么上课。']},
 '9': {'新类别': '无法总结',
  '占比': '1.99%',
  '问题': ['那每周的每周有三天的话，一天两小时，那个课时是多长时间，960分钟，那得多长时间。',
   '三次。那一节课大概是多长时间，我们是。',
   '一周二到三个小时，一个课时是多长时间？',
   '就是每周三次，然后一共课时有多少次。',
   '课时是多少天嘛，大概

In [76]:
data5_use = {}
for keys5,values5 in data4_use.items():
    dic5 = {}
    qu_vec_list5 = []
    new_class5_vec = s.encode(values['新类别'])
    for i in values5:
        qu_vec5 = s.encode(i)
        qu_vec_list5.append(qu_vec5)
    sim_list5 = []
    for i in qu_vec_list5:
        sim5 = cosine_similarity(new_class5_vec.reshape(1,-1),i.reshape(1,-1))
        sim5 = float(sim5)
        sim_list5.append(sim5)
    df5 = pd.concat([pd.Series(list(values5['问题'])),pd.Series(sim_list5)],axis=1)
    df5.columns = ['qu','sim']
    df5 = df5.sort_values(by='sim',ascending=False)
    dic5['新类别'] = values5['新类别']
    dic5['占比'] = values5['占比']
    dic5['问题'] = list(df5['qu'])[:10]
    data5_use[keys5] = dic5
data5_use

{'23 17 25': {'新类别': '一个课时的时长是多久？',
  '占比': '9.88%',
  '问题': ['一个课时是多长时间呢？',
   '多课时，一个课时多长时间？',
   '一个课时多长时间？',
   '一节课多长时间？',
   '多长时间一个课时？',
   '大概一个课时多长？',
   '节课多长时间？',
   '的课时是大概得多长时间？',
   '上课，他一节课多长时间？',
   '课时是多长时间一个？']},
 '15 14': {'新类别': '线上课程课时安排及学习方式',
  '占比': '6.74%',
  '问题': ['如果我上这个是必须要上够多的课时嘛，课时，这个课时是在什么时候上。',
   '修了多少课时，然后的话就是参加。',
   '一期课得多长时间呢？那你报下一期的来得及的。',
   '像你们一个课时的时长有多长，平均因为看一下就考，那到考试之前就是我因为我不知道你们开的课程一般正常。',
   '我还想知道一下，如果此刻报名的话，这个课时安排是怎么安排的？除了135每天上课两个小时，那么大约得持续几个月才能把这点儿课上',
   '对，那个。我是自由职业者。那个。那个是多少课时。',
   '课时要上课，上多少课时，我也是挺忙的。',
   '这些。多少个课时，这些那些，我又会担心自己记不住。这个就会。',
   '是192课时是一天有最多可以。可以。挂几个？一个课时吗？还是说一天就一个课时？',
   '我先看一下课程多少节，然后。我再考虑一下。']},
 '5': {'新类别': '无法总结',
  '占比': '6.39%',
  '问题': ['那你这样婚姻情感那种。还有那个高级成长的朱老师这种。他们那课时是分别是多少？',
   '我这我家这边不在郑州，反正你们这个不是也不需要线下，就是去上课吗？对。',
   '报考的话，它那个有没有相关的一些的课程之类的吗。',
   '考虑过说这个课时有多少，我就是说。现在不是把这个证考完了，然后就觉得其实。很多时候吧，这个证放生也没啥坏处，多学点知识对自己来说更没啥坏处那。',
   '我知道你这个婚姻和成长就多这两个，还有他的课时是多少课时来着。',
   '然后那个心理催眠师跟青少年成长指导师的那个课程，这个课程的话是一般是怎么安排。'

In [51]:
def qu_filter(data, top=10):
    s = SentenceTransformer('shibing624/text2vec-base-chinese')
    data_use = {}
    for keys, values in data.items():
        dic = {}
        qu_vec_list = []
        new_class_vec = s.encode(values['新类别'])
        for i in values:
            qu_vec = s.encode(i)
            qu_vec_list.append(qu_vec)
        sim_list = []
        for i in qu_vec_list:
            sim = cosine_similarity(new_class_vec.reshape(1, -1), i.reshape(1, -1))
            sim = float(sim)
            sim_list.append(sim)
        df = pd.concat([pd.Series(list(values['问题'])), pd.Series(sim_list)], axis=1)
        df.columns = ['qu', 'sim']
        df = df.sort_values(by='sim', ascending=False)
        dic['新类别'] = values['新类别']
        dic['占比'] = values['占比']
        if len(df['qu']) <= top:
            dic['问题'] = list(df['qu'])[:top]
            data_use[keys] = dic
        else:
            dic['问题'] = list(df['qu'])
            data_use[keys] = dic
    return data_use
def all_qu_filter(data):
    for keys in data.keys():
        data[keys]['过滤后的类别'] = qu_filter(data[keys]['过滤后的类别'])
        data[keys]['被过滤的类别'] = qu_filter(data[keys]['被过滤的类别'])
    return data

In [53]:
with open(r'C:\Users\10634\Desktop\recluster\全qu模式\result\filter3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
dic = {}
for key, value in data.items():
    data[key]['过滤后的类别'] = qu_filter(value['过滤后的类别'])
    data[key]['被过滤的类别'] = qu_filter(value['被过滤的类别'])

In [67]:
from sentence_transformers import SentenceTransformer,util
keys = list(data.keys())
dic = data[keys[0]]['过滤后的类别']
cvec = s.encode(dic['3']['新类别'])
qlist = dic['3']['问题']
vlist = s.encode(qlist)
slist = util.pytorch_cos_sim(cvec.reshape(1,-1),vlist)
slist = slist.tolist()[0] 

[0.5870484709739685,
 0.6712512373924255,
 0.7235870361328125,
 0.5997159481048584,
 0.6484341621398926,
 0.7364752292633057,
 0.6587923169136047,
 0.5306383371353149,
 0.6308863162994385,
 0.5223022699356079,
 0.6492830514907837,
 0.4881718158721924,
 0.5776634216308594,
 0.5319323539733887,
 0.5699697732925415,
 0.526148796081543,
 0.5171485543251038,
 0.48830509185791016,
 0.6432307958602905,
 0.5009669065475464,
 0.4734777808189392,
 0.667047381401062,
 0.5100071430206299,
 0.4876422882080078,
 0.5688654184341431,
 0.5076096653938293,
 0.5630600452423096,
 0.6099367141723633,
 0.6977642774581909,
 0.5662839412689209,
 0.5762038230895996,
 0.5596886873245239,
 0.5246788263320923,
 0.5662403106689453,
 0.5613534450531006,
 0.4598996937274933,
 0.5634175539016724,
 0.5418732166290283,
 0.6067187786102295,
 0.5000433921813965,
 0.6086806058883667,
 0.5254653692245483,
 0.544732928276062,
 0.5722161531448364,
 0.5710408687591553,
 0.4970088303089142,
 0.5727407932281494,
 0.519565343856

In [73]:
keys = list(data.keys())
keys

['52536-是什么原因？',
 '52544-费用相关问题',
 '52553-请给我一些考虑时间好吗？',
 '52555-线上线下课程设置、考试方式及学习方式简介',
 '52557-有关证书的问题',
 '52558-报名、报考、考试时间及方式止，如何报名，何处报名？',
 '52560-如何通过微信联系？',
 '54230-第一个班型的课程时长及时间',
 '54238-大概是多少',
 '54256-如何选择考试方向？',
 '54646-学习和考试周期需要多长时间',
 '54647-发信息给我',
 '55308-有可能性吗？',
 '55951-是些什么内容呢？',
 '56520-怎么收费',
 '56529-这个考试好通过吗']

In [7]:
import json
import os
from sentence_transformers import SentenceTransformer,util
import sys
import json
import pandas as pd
def filter_qu(data,top=10):
    s = SentenceTransformer('shibing624/text2vec-base-chinese')
    for keys,values in data.items():
        class_vec = s.encode(data[keys]['新类别'])
        qu_list = data[keys]['问题']
        vec_list = s.encode(qu_list)
        sim_list = util.pytorch_cos_sim(class_vec.reshape(1,-1),vec_list)
        sim_list = sim_list.tolist()[0]
        df = pd.concat([pd.Series(qu_list),pd.Series(sim_list)],axis=1)
        df.columns = ['qu','sim']
        df = df.sort_values(by='sim',ascending=False)
        data[keys]['问题'] = list(df['qu'])[:top]
    return data

In [11]:
with open(r'C:\Users\10634\Desktop\recluster\半qu模式\result\ord_filter3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
for keys in data.keys():
    print(keys)
    data[keys]['过滤后的类别'] = filter_qu(data[keys]['过滤后的类别'])
    data[keys]['被过滤的类别'] = filter_qu(data[keys]['被过滤的类别'])
with open(r'C:\Users\10634\Desktop\recluster\半qu模式\result\filter3_top10.json', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(data, ensure_ascii=False, indent=4, separators=(',', ':')))
data

52544-费用相关问题
52558-报名、报考、考试时间及方式止，如何报名，何处报名？
54238-大概是多少
52557-有关证书的问题
52560-如何通过微信联系？
52555-线上线下课程设置、考试方式及学习方式简介
54256-如何选择考试方向？
52553-请给我一些考虑时间好吗？
52536-是什么原因？
55951-是些什么内容呢？
55308-有可能性吗？
54647-发信息给我
54230-第一个班型的课程时长及时间
54646-学习和考试周期需要多长时间
56520-怎么收费
56529-这个考试好通过吗


{'52544-费用相关问题': {'过滤后的类别': {'44': {'新类别': '报名费用是多少？',
    '占比': '14.78%',
    '问题': ['报名费用是多少？',
     '报名费用是多少？',
     '报名的费用是多少？',
     '报名的费用是多少？',
     '报名费用大概多少？',
     '报名费用是大概是多少？',
     '报名费是多少钱？',
     '报名费是多少钱？',
     '报名费是多少钱？',
     '报名费是多少钱？']},
   '1': {'新类别': '费用多少？',
    '占比': '6.93%',
    '问题': ['费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？',
     '费用是多少？']},
   '35': {'新类别': '收费情况如何？',
    '占比': '3.39%',
    '问题': ['收费是怎么样的？',
     '收费是什么情况？',
     '怎么样的收费情况？',
     '收费是什么样的？',
     '好，收费情况是怎么样的？',
     '收费是怎么样的呢？',
     '收费是怎么收的？',
     '收费是怎么收的？',
     '收费是怎么收的？',
     '收费是怎么收的？']},
   '18': {'新类别': '学费多少钱？',
    '占比': '2.52%',
    '问题': ['学费大概是多少钱？',
     '学费大概是多少？',
     '大概学费要多少钱？',
     '具体学费多少钱？',
     '大概学费要多少？',
     '这个学费多少？',
     '这个学费是多少钱？',
     '这个学费大概是多少钱？',
     '大概学费是多少呢？',
     '这个学费是多少钱。']},
   '30': {'新类别': '心理咨询师考试报名费用及流程',
    '占比': '2.38%',
    '问题': ['如果要考那个心理咨询师